##### 对于覆盖率小于10%的特征，判断是否为空对成交率的影响

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [8]:
wide_info_others = pd.read_csv('/home/mam_jupyter/jupyter_dir/artefact/leads_scoring_model/wide_info_others.csv', delimiter = ',')
cov_rate = pd.read_csv('/home/mam_jupyter/jupyter_dir/artefact/leads_scoring_model/EDA/cov_rate.csv')

/app/anaconda3/envs/python27_smcsandbox/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
# select all columns with cov_rate smaller than 10%
wide_null_col = wide_info_others[['mobile', 'd_deal_flag'] + list(cov_rate[(cov_rate['cov_rate_others'] <= 10) & \
                                                                           (cov_rate['index'].str.contains('h_') == False)]['index'])]

In [13]:
wide_null_col.columns

Index([u'mobile', u'd_deal_flag', u'd_fir_sec_visit_diff',
       u'd_last_reservation_time', u'd_trail_attend_ttl',
       u'd_trail_attend_ppt', u'd_dealf_succ_firvisit_diff',
       u'd_dealf_succ_lastvisit_diff', u'd_avg_fir_sec_visit_diff',
       u'd_avg_firleads_firvisit_diff', u'd_fir_activity_time',
       u'd_fir_leads_deal_diff_y', u'd_trail_count_d30', u'd_trail_count_d90',
       u'd_visit_count_d15', u'd_activity_count_d15', u'd_activity_count_d30',
       u'd_activity_count_d60', u'd_activity_count_d90',
       u'd_firfollow_dealf_diff', u'd_lastfollow_dealf_diff',
       u'd_firleads_firvisit_diff', u'd_leads_dtbt_ppt',
       u'd_fir_dealfail_deal_diff', u'd_last_activity_dealf_diff',
       u'd_fir_activity_dealf_diff', u'd_fir_order_leads_diff',
       u'd_fir_order_visit_diff', u'd_fir_order_trail_diff',
       u'c_last_sis_time', u'c_register_time'],
      dtype='object')

In [60]:
null_col  = list(cov_rate[(cov_rate['cov_rate_others'] <= 10) & (cov_rate['index'].str.contains('h_') == False)]['index'])

# transfer values in a column into binary values given none or not
wide_null_col[null_col] = wide_null_col[null_col].where(wide_null_col[null_col].isnull(),1).fillna(0).astype(int)

/app/anaconda3/envs/python27_smcsandbox/lib/python2.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [20]:
def compute_lift_matrix(df, action_col, conv_col):
    '''
    Functions that compute a lift matrix given:
    df (dataframe): input dataframe
    conv_col (string): column name of the dataframe containing the purchase result
    action_col (list of string): list of column name of the dataframe containing the actions
    '''
    list_conversion = [0, 1]
    lift_matrix_to_df = [] #list of lists from which we create the lift matrix
    action_index = [] #future indexes of the lift matrix  
    
    def compute_prob_cond(df, conv_col, conversion_item, action, action_instance):
        prob_cond = float(df[(df[conv_col] == conversion_item) & (df[action] == action_instance)]\
                          ["mobile"].unique().shape[0])/float(df[df[action] == action_instance]["mobile"].unique().shape[0])
        return prob_cond

    def compute_prob_conv(df, conv_col, conversion_item):
        prob_conv = float(df[df[conv_col] == conversion_item]["mobile"].unique().shape[0])/float(df["mobile"].unique().shape[0])
        return prob_conv

    def compute_lift(prob_cond, prob_conv):
        return prob_cond/prob_conv
    
    #We iterate over the different action columns we want to see in the lift matrix
    for action in action_col:
        list_actions = [0,1] #left column in the matrix
        action_list_name = [str(action) + "_" + str(a) for a in list_actions]
        action_index = action_index + action_list_name #names of the actions that will appear on the lift matrix
        for action_instance in list_actions:
            lift_row = [] #row with lifts, row to be added successively to the matrix 
            for conversion_item in list_conversion:
                try:
                    prob_cond = compute_prob_cond(df, conv_col, conversion_item, action, action_instance)
                    prob_conv = compute_prob_conv(df, conv_col, conversion_item)
                    prob_cond = compute_lift(prob_cond, prob_conv)
                except ZeroDivisionError:
                    prob_cond = "NA"
                lift_row.append(prob_cond)
            lift_row.append(df[df[action] == action_instance]["mobile"].unique().shape[0]) # add total population
            lift_matrix_to_df.append(lift_row)
    # create the lift matrix
    lift_matrix = pd.DataFrame(lift_matrix_to_df, columns = list_conversion + ['Total action population'])
    lift_matrix["action"] = action_index
    lift_matrix = lift_matrix.set_index("action", drop = True)
    
    return lift_matrix

In [62]:
lift_matrix_if_none = compute_lift_matrix(wide_null_col, null_col , 'd_deal_flag')

In [64]:
lift_matrix_if_none.to_csv('lift_matrix_if_none.csv')